# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 76 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.08790


extracting tarball to tmp_2412.08790... done.


/tmp/ipykernel_2679/2822249172.py:52: LatexWarning: 2412.08790 did not run properly
string index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2412.08877


...

extracting tarball to tmp_2412.09297... done.
Retrieving document from  https://arxiv.org/e-print/2412.09363


extracting tarball to tmp_2412.09363...

 done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 87 bibliographic references in tmp_2412.09363/aanda.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09363-b31b1b.svg)](https://arxiv.org/abs/2412.09363) | **Behind the dust veil: A panchromatic view of an optically dark galaxy at z=4.82**  |
|| N. B. Sillassen, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-12-13*|
|*Comments*| *11 pages, 7 figures, accepted in Astronomy & Astrophysics*|
|**Abstract**|            Optically dark dusty star-forming galaxies (DSFGs) play an essential role in massive galaxy formation at early cosmic time, however their nature remains elusive. Here we present a detailed case study of all the baryonic components of a $z=4.821$ DSFG, XS55. Selected from the ultra-deep COSMOS-XS 3GHz map with a red SCUBA-2 450$\mu$m/850$\mu$m colour, XS55 was followed up with ALMA 3mm line scans and spectroscopically confirmed to be at $z=4.821$ via detections of the CO(5-4) and [CI](1-0) lines. JWST/NIRCam imaging reveals that XS55 is a F150W-dropout with red F277W/F444W colour, and a complex morphology: a compact central component embedded in an extended structure with a likely companion. XS55 is tentatively detected in X-rays with both Chandra and XMM-Newton, suggesting an active galactic nucleus (AGN) nature. By fitting a panchromatic SED spanning NIR to radio wavelengths, we revealed that XS55 is a massive main-sequence galaxy with a stellar mass of $M_\ast=(5\pm1)\times10^{10}\,{\rm M_\odot}$ and a star formation rate of ${\rm SFR}=540\pm177~{\rm M_\odot\,yr^{-1}}$. The dust of XS55 is optically thick in the far infrared (FIR) with a surprisingly cold dust temperature of $T_{\rm dust}=33\pm2\,{\rm K}$, making XS55 one of the coldest DSFGs at $z>4$ known to date. This work unveils the nature of a radio-selected F150W-dropout, suggesting the existence of a population of DSFGs hosting active black holes embedded in optically thick dust.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08877-b31b1b.svg)](https://arxiv.org/abs/2412.08877) | **Discoveries and Properties of EL CVn-type Binaries in the TESS Survey**  |
|| J. Xiong, et al. -- incl., <mark>J. Li</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-13*|
|*Comments*| *13 pages, 10 figures, 2 tables*|
|**Abstract**|            EL CVn-type systems represent a rare evolutionary stage in binary star evolution, providing ideal laboratories for investigating stable mass transfer processes and the formation of extremely low-mass white dwarfs (ELM WDs). The Transiting Exoplanet Survey Satellite (TESS) has delivered an extensive collection of high-precision time-domain photometric data, which is invaluable for studying EL CVn binaries. In this study, we identified 29 EL CVn systems from the TESS eclipsing binary catalogs (sectors 1-65), 11 of which are newly discovered. These systems consist of smaller, hotter pre-He white dwarfs and A/F main-sequence stars. The orbital periods of these binaries range from 0.64 to 2.5 days. Utilizing TESS light curves, Gaia distances, and multi-band photometric data (e.g., GALEX, 2MASS, WISE, SkyMapper), we modeled the light curves and spectral energy distributions to derive system parameters, including effective temperatures, masses, and radii. These systems were then compared with the white dwarf mass-period relation and the evolutionary tracks of ELM WDs. The comparison reveals that these binaries are consistent with the expected mass-period relation for white dwarfs and align well with the evolutionary tracks on the Teff-logg diagram for ELM WDs. This result suggests that these EL CVn systems likely formed through stable mass transfer processes. We provide a catalog of complete parameters for 29 EL CVn systems identified from the TESS survey. This catalog will serve as an essential resource for studying binary mass transfer, white dwarf formation, and pulsation phenomena in EL CVn-type systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08903-b31b1b.svg)](https://arxiv.org/abs/2412.08903) | **Binary Evolution Pathways to Blue Large-Amplitude Pulsators: Insights from HD 133729**  |
|| Z. Zhang, C. Wu, <mark>X. Zhang</mark>, B. Wang |
|*Appeared on*| *2024-12-13*|
|*Comments*| *Comments are welcome*|
|**Abstract**|            Blue Large-Amplitude Pulsators (BLAPs) represent a recently identified class of pulsating stars distinguished by their short pulsation periods (22-40 minutes) and asymmetric light curves. This study investigates the evolutionary channel of HD 133729, the first confirmed BLAP in a binary system. Through binary evolution simulations with MESA, we explored various mass ratios and initial orbital periods, finding that a mass ratio of q = 0.30 coupled with non-conservative mass transfer ($\rm \beta$ = 0.15) successfully reproduces the observational characteristics (including luminosity, surface gravity, and effective temperature) of the binary system. Our models not only match the observed pulsational properties but also predict significant helium and nitrogen enhancements on the surface of the main-sequence companion. The system is expected to eventually undergo a common envelope phase leading to a stellar merger. Our findings provide crucial insights into the formation mechanism and evolutionary fate of BLAPs with main-sequence companions, while also placing constraints on the elemental abundances of their binary companions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09297-b31b1b.svg)](https://arxiv.org/abs/2412.09297) | **Constraining primordial black hole abundance with Insight-HXMT**  |
|| C. Yang, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-13*|
|*Comments*| *10 pages, 2 figures*|
|**Abstract**|            Primordial black holes (PBHs) are a major candidate for dark matter and they have been extensively constrained across most mass ranges. However, PBHs in the mass range of $10^{17}-10^{21}$ g remain a viable explanation for all dark matter. In this work, we use observational data from the Hard X-ray Modulation Telescope (Insight-HXMT) to refine constraints on PBHs within the mass range of $10^{16}-10^{18}$ g. Our analysis explores three scenarios: directly using observational data, incorporating the astrophysical background model (ABM), and employing the power-law spectrum with an exponential cutoff. Additionally, we also account for photon flux from positron annihilation due to Hawking radiation in our calculations. Our findings indicate that directly using observational data significantly tightens constraints on PBHs with masses above $10^{17}$ g, owing to Insight-HXMT's exceptional sensitivity to hard X-rays. Incorporating the ABM further enhances the constraints by approximately 0.8 orders of magnitude. The exclusion limit for PBHs as dark matter extends to $8\times 10^{17}$ g, nearly doubling the current threshold. Moreover, extrapolating the energy limit with a power-law model improves constraints by approximately one order of magnitude compared to the ABM scenario.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08790-b31b1b.svg)](https://arxiv.org/abs/2412.08790) | **Magnetic Fields in Massive Star-forming Regions (MagMaR). V. The Magnetic Field at the Onset of High-mass Star Formation**  |
|| P. Sanhueza, et al. -- incl., <mark>J. Liu</mark>, <mark>H. Beuther</mark> |
|*Appeared on*| *2024-12-13*|
|*Comments*| *Accepted for publications in ApJ (9 pages, 3 figures, Appendix)*|
|**Abstract**|            A complete understanding of the initial conditions of high-mass star formation and what processes determine multiplicity require the study of the magnetic field (B-field) in young, massive cores. Using ALMA 250 GHz polarization (0.3" = 1000 au) and ALMA 220 GHz high-angular resolution observations (0.05" = 160 au), we have performed a full energy analysis including the B-field at core scales and have assessed what influences the multiplicity inside a massive core previously believed to be in the prestellar phase. With 31 Msun, the G11.92 MM2 core has a young CS outflow with a dynamical time scale of a few thousand years. At high-resolution, the MM2 core fragments into a binary system with a projected separation of 505 au and a binary mass ratio of 1.14. Using the DCF method with an ADF analysis, we estimate in this core a B-field strength of 6.2 mG and a mass-to-flux ratio of 18. The MM2 core is strongly subvirialized with a virial parameter of 0.064, including the B-field. The high mass-to-flux ratio and low virial parameter indicate that this massive core is very likely undergoing runaway collapse, which is in direct contradiction with the core-accretion model. The MM2 core is embedded in a filament that has a velocity gradient consistent with infall. In line with clump-fed scenarios, the core can grow in mass at a rate of 1.9--5.6 x 10^-4 Msun/yr. In spite of the B-field having only a minor contribution to the total energy budget at core scales, it likely plays a more important role at smaller scales by setting the binary properties. Considering energy ratios and a fragmentation criterion at the core scale, the binary could have been formed by core fragmentation. The binary properties (separation and mass ratio), however, are also consistent with radiation-magnetohydrodynamic simulations with super-Alfvenic, supersonic (or sonic) turbulence that form binaries by disk fragmentation.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error string index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.09363/./XS55_spectrum_full.png', 'tmp_2412.09363/./lines_velocity.png', 'tmp_2412.09363/./XS55_cont_image.png', 'tmp_2412.09363/./XS55_CO54_image.png', 'tmp_2412.09363/./XS55_CI10_image.png', 'tmp_2412.09363/./XS55_galfit_full.png', 'tmp_2412.09363/./60000055.png']
copying  tmp_2412.09363/./XS55_spectrum_full.png to _build/html/
copying  tmp_2412.09363/./lines_velocity.png to _build/html/
copying  tmp_2412.09363/./XS55_cont_image.png to _build/html/
copying  tmp_2412.09363/./XS55_CO54_image.png to _build/html/
copying  tmp_2412.09363/./XS55_CI10_image.png to _build/html/
copying  tmp_2412.09363/./XS55_galfit_full.png to _build/html/
copying  tmp_2412.09363/./60000055.png to _build/html/
exported in  _build/html/2412.09363.md
    + _build/html/tmp_2412.09363/./XS55_spectrum_full.png
    + _build/html/tmp_2412.09363/./lines_velocity.png
    + _build/html/tmp_2412.09363/./XS55_cont_image.png
    + _build/html/tmp_2412.09363/./XS55_CO54_image.png
    + _build/html/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\changes}[1]{\textcolor{purple}{\bf #1}}$
$\newcommand{\nochanges}[1]{#1}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Behind the dust veil: A panchromatic view of an optically dark galaxy at z=4.82

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.09363-b31b1b.svg)](https://arxiv.org/abs/2412.09363)<mark>Appeared on: 2024-12-13</mark> -  _11 pages, 7 figures, accepted in Astronomy & Astrophysics_

</div>
<div id="authors">

N. B. Sillassen, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Optically dark dusty star-forming galaxies (DSFGs) play an essential role in massive galaxy formation at early cosmic time, however their nature remains elusive.Here we present a detailed case study of all the baryonic components of a $z=4.821$ DSFG, XS55. Selected from the ultra-deep COSMOS-XS 3GHz map with a red SCUBA-2 450 $\mu$ m/850 $\mu$ m colour, XS55 was followed up with ALMA 3mm line scans and spectroscopically confirmed to be at $z=4.821$ via detections of the CO(5-4) and [ C ${\tiny I}$ ] (1-0) lines.JWST/NIRCam imaging reveals that XS55 is a F150W-dropout with red F277W/F444W colour, and a complex morphology: a compact central component embedded in an extended structure with a likely companion.XS55 is tentatively detected in X-rays with both Chandra and XMM-Newton, suggesting an active galactic nucleus (AGN) nature. By fitting a panchromatic SED spanning NIR to radio wavelengths, we revealed that XS55 is a massive main-sequence galaxy with a stellar mass of $M_\ast=(5\pm1)\times10^{10} {\rm M_\odot}$ and a star formation rate of ${\rm SFR}=540\pm177 {\rm M_\odot yr^{-1}}$ . The dust of XS55 is optically thick in the far infrared (FIR) with a surprisingly cold dust temperature of $T_{\rm dust}=33\pm2 {\rm K}$ , making XS55 one of the coldest DSFGs at $z>4$ known to date. This work unveils the nature of a radio-selected F150W-dropout, suggesting the existence of a population of DSFGs hosting active black holes embedded in optically thick dust.

</div>

<div id="div_fig1">

<img src="tmp_2412.09363/./XS55_spectrum_full.png" alt="Fig3.1" width="20%"/><img src="tmp_2412.09363/./lines_velocity.png" alt="Fig3.2" width="20%"/><img src="tmp_2412.09363/./XS55_cont_image.png" alt="Fig3.3" width="20%"/><img src="tmp_2412.09363/./XS55_CO54_image.png" alt="Fig3.4" width="20%"/><img src="tmp_2412.09363/./XS55_CI10_image.png" alt="Fig3.5" width="20%"/>

**Figure 3. -** ** Top:** ALMA 3mm spectrum of XS55. The red line shows the line-free continuum, and the blue dotted line indicates the flux error per channel at $1\sigma$ level. The spectroscopic redshift is shown in text, along with the velocity width of the channels. ** Bottom-left:** Velocity space spectrum of CO(5-4) (blue) and [C{\tiny I}](1-0) (red) at $z=4.8214$, the uncertainty per channel is shown as dashed lines. ** Bottom-right:** Continuum and continuum subtracted moment-0 line maps of XS55. Contour levels are 5, 8, and 11$\sigma$ for continuum, and 3, 4, 5$\sigma$ for the line maps. The beam size is shown as a white ellipse, and the peak JWST/F444W position is marked by a grey cross. (*fig:spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2412.09363/./XS55_galfit_full.png" alt="Fig1" width="100%"/>

**Figure 1. -** Morphological fit of XS55 in JWST/F444W (top and middle row) and JWST/F277W (bottom row). The different components of each fit are stated in the right column of each row. (*fig:morph*)

</div>
<div id="div_fig3">

<img src="tmp_2412.09363/./60000055.png" alt="Fig2" width="100%"/>

**Figure 2. -** Multi-wavelength cutout images of XS55. The instrument, wavelength and field of view (FoV) are shown in green text in each panel. (*fig:cutouts*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.09363"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

281  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
